In [14]:
messages = [line.rstrip() for line in open('smsspamcollection/SMSSpamCollection')]
print(len(messages))

5574


In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
for message_no,message in enumerate(messages[:10]):
    print(message_no,message)

0 ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
1 ham	Ok lar... Joking wif u oni...
2 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3 ham	U dun say so early hor... U c already then say...
4 ham	Nah I don't think he goes to usf, he lives around here though
5 spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv
6 ham	Even my brother is not like to speak with me. They treat me like aids patent.
7 ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
8 spam	WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 1

In [17]:
messages_df = pd.read_csv('smsspamcollection/SMSSpamCollection',sep = '\t',names = ('label','message'))

In [18]:
messages_df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords

In [45]:
def text_process(mess):
    
    nopunc_df = [char for char in mess if char not in string.punctuation]
    
    nopunc_df = ''.join(nopunc_df)
    
    return [word for word in nopunc_df.split() if word.lower() not in stopwords.words('english')]

In [46]:
messages_df['message'].head(5).apply(text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [47]:
bow_transformer = CountVectorizer(analyzer = text_process).fit(messages_df['message'])

In [48]:
bow_vector = bow_transformer.transform(messages_df['message'])

In [63]:
print ('The shape of the sparse matrix is : '), print(bow_vector.shape)

The shape of the sparse matrix is : 
(5572, 11425)


(None, None)

In [64]:
print('The no of non-zero occurences in the vector is :'),print(bow_vector.nnz)

The no of non-zero occurences in the vector is :
50548


(None, None)

In [72]:
print('the sparsity of the vector:'), print((bow_vector.nnz*100)/(bow_vector.shape[0]*bow_vector.shape[1]))

the sparsity of the vector:
0.07940295412668218


(None, None)

In [81]:
message4 = (messages_df['message'][3])
print(message4)

U dun say so early hor... U c already then say...


In [83]:
print(bow_transformer.get_feature_names()[4039])

Truth


In [84]:
print(bow_transformer.get_feature_names()[4073])

UIN


In [74]:
from sklearn.feature_extraction.text import TfidfTransformer

In [86]:
tfidf = TfidfTransformer().fit(bow_vector)

In [87]:
tfidftransform = tfidf.transform(bow_vector)

In [88]:
tfidftransform.shape

(5572, 11425)

In [89]:
from sklearn.naive_bayes import MultinomialNB

In [91]:
spam_detection = MultinomialNB().fit(tfidftransform,messages_df['label'])

In [92]:
predictions = spam_detection.predict(tfidftransform)

In [93]:
predictions

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [94]:
from sklearn.metrics import classification_report,confusion_matrix

In [96]:
print(classification_report(messages_df['label'],predictions))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      4825
        spam       1.00      0.85      0.92       747

    accuracy                           0.98      5572
   macro avg       0.99      0.92      0.95      5572
weighted avg       0.98      0.98      0.98      5572



In [97]:
from sklearn.model_selection import train_test_split

In [98]:
X_train,X_test,y_train,y_test = train_test_split(messages_df['message'],messages_df['label'],test_size = 0.3)

In [101]:
from sklearn.pipeline import Pipeline

In [107]:
pipeline = Pipeline([
    ('bow_vector',CountVectorizer(analyzer=text_process)),
    ('tfidftransform',TfidfTransformer()),
    ('spam_classifier',MultinomialNB()),
])

In [108]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('bow_vector',
                 CountVectorizer(analyzer=<function text_process at 0x0000015506F36700>)),
                ('tfidftransform', TfidfTransformer()),
                ('spam_classifier', MultinomialNB())])

In [109]:
predictions = pipeline.predict(X_test)

In [110]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1453
        spam       1.00      0.74      0.85       219

    accuracy                           0.97      1672
   macro avg       0.98      0.87      0.91      1672
weighted avg       0.97      0.97      0.96      1672

